# Aprendizaje Automático (clase 3)

## Diego Fernando Agudelo 
## Universidad ICESI 
## diegoagudelo30@gmail.com

## **1. Carga de paquetes**

#!pip list | grep numpy
#!pip install "numpy==1.26.4"
#!pip install "pmdarima==2.0.4"
!pip install "pandas"
!pip install "statsmodels"
!pip install "scikit-learn"
!pip install "matplotlib"
!pip install "seaborn"
!pip install "openpyxl"


In [ ]:
#!pip install "numpy==1.26.4"
#!pip install "pmdarima==2.0.4"

In [ ]:
#!pip install pmdarima
from pmdarima.arima import auto_arima
from pmdarima.utils import acf,pacf
from pmdarima.arima import ARIMA
import pandas as pd
import numpy as np 
from statsmodels.sandbox.stats.runs import runstest_1samp # prueba de rachas de Wald y Wolfowitz
import statsmodels.api as sm # prueba de Box-Pierce y la modificación de Ljung-Box
from matplotlib import pyplot as plt # gráficos 
import pylab as py
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

: 

## **2. Carga de datos**

Vamos a usar unos datos simulados de un proceso ARIMA(0,1,1)

In [ ]:
data = pd.read_excel("https://raw.githubusercontent.com/profedaniel86/Series_de_Tiempo/refs/heads/main/1.Intro/datosEmpleo.xlsx",index_col='mes',parse_dates=True)
data.head()

data = pd.read_csv("./arima011.csv")
data.head()

## **3. ACF y PACF**

In [ ]:
acf(data['Ocupados'],40)

In [ ]:
plot_acf(data['Ocupados'],lags=100);
plot_pacf(data['Ocupados'],lags=100);

In [ ]:
## information_criterion (‘aic’, ‘aicc’, ‘bic’, ‘hqic’, ‘oob’)
model = auto_arima(data["Ocupados"],max_p=10, max_q=10,information_criterion = ("aic"))
model.summary()

In [ ]:
print(model.resid())

In [ ]:
data['Ocupados'].plot()

In [ ]:
fig = plt.figure(figsize=(12, 6))
plt.plot(model.resid(),label="Residuales")

### Debido a qque el primer registro no tiene como compararse con datos anteriores, el sistema rellena los datos con valores simulados.
Se decide graficar sin el primer registro de los residuos.

In [ ]:
fig = plt.figure(figsize=(12, 6))
plt.plot(model.resid()[1:],label="Residuales")

## **4. Pronóstico del ARIMA(0,1,1) obtenido en el auto.arima**


In [ ]:
h=10

y_forec, conf_int  = model.predict(h,return_conf_int=True,alpha=0.05)
preds = pd.DataFrame(np.column_stack([y_forec,conf_int[:,0] , conf_int[:,1]]))
preds.columns = ['Point_forecast', 'lower_95', 'upper_95']

x_for= np.linspace((data.shape[0]+1),(data.shape[0]+h),h)

fig = plt.figure(figsize=(12, 6))
plt.plot(data.index,data["Ocupados"],label="Ocupados")
plt.plot(x_for,preds['Point_forecast'],label="Forecast Ocupados")
plt.fill_between(x_for,preds['lower_95'], preds['upper_95'], color='blue', alpha=0.1)
plt.legend()
plt.show()


## **5. Replicando el modelo sin hacer auto arima**

In [ ]:
model = ARIMA(order=(0,1,1))
results = model.fit( data["ARIMA011"])
results.summary()

## **6. Intervalos de confianza usando boostrap**


In [ ]:
h= 10
res = results.resid()

my_samples = []
for _ in range(1000):

    x = np.random.choice(res, size=h, replace=True)
    my_samples.append(x)

q1=np.quantile(my_samples,0.025,axis=0)
q2=np.quantile(my_samples,0.975,axis=0) 

fore = results.predict(h,return_conf_int=False)

preds = pd.DataFrame(np.column_stack([fore,fore+np.cumsum(q1) , fore+np.cumsum(q2)]))
preds.columns = ['Point_forecast', 'lower_95', 'upper_95']

preds


In [ ]:
x_for= np.linspace((data.shape[0]+1),(data.shape[0]+h),h)

fig = plt.figure(figsize=(12, 6))
plt.plot(data["time"],data["ARIMA011"],label="ARIMA011")
plt.plot(x_for,preds['Point_forecast'],label="Forecast Boostrap")
plt.fill_between(x_for,preds['lower_95'], preds['upper_95'], color='blue', alpha=0.1)
plt.legend()
plt.show()

### **7. Ejercicio en Clase**

Empleando la información del número de ocupados en miles de personas (Ocupados) para las 13 principales ciudades, encuentre el mejor pronóstico para los próximos 6 meses empleando los métodos vistos en la clase. Compare
los resultados con el mejor modelo encontrado en el ejercicio anterior. 

Escriba un breve informe de máximo una página de texto que explique cómo llega a sus proyeccciones y presente las proyecciones. Aclare en el texto cuáles serían las limitaciones de sus pronósticos.